In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import scipy.optimize

# Define the Physics-Informed Neural Network (PINN)
class PINN(keras.Model):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.model = self.build_model(layers)
    
    def build_model(self, layers):
        model = keras.Sequential()
        model.add(keras.layers.InputLayer(input_shape=(4,)))  # (x, y, z, t)
        for width in layers[1:-1]:
            model.add(keras.layers.Dense(width, activation='tanh'))
        model.add(keras.layers.Dense(layers[-1], activation=None))  # Output T
        return model
    
    def call(self, x):
        return self.model(x)

# Define heat equation residual
@tf.function
def heat_residual(model, x, y, z, t, rho, cp, k, Q):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, y, z, t])
        inputs = tf.stack([x, y, z, t], axis=1)
        T = model(inputs)
        
        T_t = tape.gradient(T, t)
        T_x = tape.gradient(T, x)
        T_y = tape.gradient(T, y)
        T_z = tape.gradient(T, z)
        
        T_xx = tape.gradient(T_x, x)
        T_yy = tape.gradient(T_y, y)
        T_zz = tape.gradient(T_z, z)
    
    del tape
    
    residual = rho * cp * T_t - k * (T_xx + T_yy + T_zz) - Q
    return residual

# Generate training data (spatial + temporal domain)
def generate_training_data(num_points):
    x = np.random.uniform(0, 0.01, num_points)
    y = np.random.uniform(0, 0.01, num_points)
    z = np.random.uniform(0, 0.005, num_points)
    t = np.random.uniform(0, 0.01, num_points)
    return x, y, z, t

# Physics parameters
rho = 7850      # kg/m³ (steel)
cp = 500        # J/(kg*K)
k = 25          # W/(m*K)
Q = 1e6         # Gaussian laser heat source (simplified)

# Training data
num_points = 5000
x_train, y_train, z_train, t_train = generate_training_data(num_points)

# Convert to TensorFlow tensors
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
z_train = tf.convert_to_tensor(z_train, dtype=tf.float32)
t_train = tf.convert_to_tensor(t_train, dtype=tf.float32)

# Instantiate and compile the model
pinn = PINN(layers=[4, 50, 50, 50, 1])  # Input 4 (x,y,z,t) -> 3 hidden layers -> Output 1 (T)
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Custom training loop
def train_step():
    with tf.GradientTape() as tape:
        residuals = heat_residual(pinn, x_train, y_train, z_train, t_train, rho, cp, k, Q)
        loss = tf.reduce_mean(tf.square(residuals))
    gradients = tape.gradient(loss, pinn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, pinn.trainable_variables))
    return loss

# Train the model
epochs = 5000
for epoch in range(epochs):
    loss = train_step()
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Save the trained model
pinn.save('pinn_heat_slm.h5')


Epoch 0, Loss: 58769354752.0
Epoch 500, Loss: 712.186279296875
Epoch 1000, Loss: 847.318603515625
Epoch 1500, Loss: 557.3079223632812
Epoch 2000, Loss: 1356828.0
Epoch 2500, Loss: 16987.09765625
Epoch 3000, Loss: 16094.3427734375
Epoch 3500, Loss: 16448.322265625
Epoch 4000, Loss: 16177.8203125
Epoch 4500, Loss: 12546.599609375


NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.

In [2]:

# Save the trained model
pinn.save_weights('pinn_heat_slm_weights.h5')


In [4]:
# Load and use the model for prediction
def predict_temperature(x, y, z, t):
    input_tensor = tf.convert_to_tensor([[x, y, z, t]], dtype=tf.float32)
    return pinn(input_tensor).numpy()[0]

In [5]:
x_test, y_test, z_test, t_test = 0.005, 0.005, 0.002, 0.01
predicted_T = predict_temperature(x_test, y_test, z_test, t_test)
print(f"Predicted temperature at ({x_test}, {y_test}, {z_test}, {t_test}): {predicted_T} K")

Predicted temperature at (0.005, 0.005, 0.002, 0.01): [0.01563277] K


In [1]:
# Re-import necessary libraries after execution state reset
import numpy as np

# Define simulation parameters
timestep_value = 15000  # Given TIMESTEP
timestep_size = 1.25e-10  # Timestep in seconds
omega = 30000  # Laser oscillation frequency (rad/s)

# Compute actual time at TIMESTEP 15000
time_at_15000 = timestep_value * timestep_size

# Compute laser coordinates at this time
a_at_15000 = 27 * time_at_15000 + 0.0075 * np.cos(omega * time_at_15000 + np.pi) + 0.0125
b_at_15000 = 0.0075 * np.sin(omega * time_at_15000 + np.pi)

# Display results
a_at_15000, b_at_15000


(0.00506248710617635, -0.0004216525620487625)

In [7]:

# Load and use the model for prediction
def predict_temperature(x, y, z, v):
    input_tensor = tf.convert_to_tensor([[x, y, z, v]], dtype=tf.float32)
    return float(pinn(input_tensor).numpy()[0])

# Predict temperature around a given point
def predict_surrounding_area(x_center, y_center, z_center, v, step=0.001, grid_size=5):
    predictions = []
    for dx in np.linspace(-step, step, grid_size):
        for dy in np.linspace(-step, step, grid_size):
            for dz in np.linspace(-step, step, grid_size):
                x, y, z = x_center + dx, y_center + dy, z_center + dz
                T_pred = predict_temperature(x, y, z, v)
                predictions.append((x, y, z, v, T_pred))
    return predictions

# Example usage: Predict temperature around a specific point
x_center, y_center, z_center, v = 0.005, 0.005, 0.002, 1.0
predictions_surrounding = predict_surrounding_area(x_center, y_center, z_center, v)

# Print predictions for surrounding area
for p in predictions_surrounding:
    print(f"Surrounding Area - x={p[0]:.4f}, y={p[1]:.4f}, z={p[2]:.4f}, v={p[3]:.2f}, T={p[4]:.2f} K")

Surrounding Area - x=0.0040, y=0.0040, z=0.0010, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0040, z=0.0015, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0040, z=0.0020, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0040, z=0.0025, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0040, z=0.0030, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0045, z=0.0010, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0045, z=0.0015, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0045, z=0.0020, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0045, z=0.0025, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0045, z=0.0030, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0050, z=0.0010, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0050, z=0.0015, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0050, z=0.0020, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0050, z=0.0025, v=1.00, T=0.26 K
Surrounding Area - x=0.0040, y=0.0050, z=0.0030, v=1.00, T=0.26 K
Surroundin

In [17]:
a=0

In [18]:
for i in range(num_files):
    a+=1
    

In [19]:
a

241

In [20]:
for i in range(241):  # Assuming 241 time steps
    file_name = f"{file_prefix}{i * timestep_interval}.dump"
    full_path = os.path.join(data_dir, file_name)
    
    if file_name not in existing_files:
        print(f"Skipping missing file: {file_name}")
        continue
    
    with open(full_path, "r") as file:
        lines = file.readlines()
    
    start_reading = False
    for line in lines:
        if "ITEM: ATOMS" in line:
            start_reading = True
            continue
        
        if start_reading:
            parts = line.split()
            if len(parts) == 6:
                x, y, z, temp = map(float, parts[2:6])
                t = i * timestep_interval * timestep_size  # Compute actual time
                data_points.append([x, y, z, t, temp])

NameError: name 'existing_files' is not defined

In [21]:
# Directory containing the dump files
data_dir = r"C:\Users\MAY02\Desktop\SLM_NeuralNetwork\Relax_Al_helix_1_1"
file_prefix = "coord"
timestep_interval = 100000  # Time step between each file
timestep_size = 1.25e-10  # Each step duration in seconds

# Get all existing dump files in the directory
existing_files = set(os.listdir(data_dir))
data_points = []

print("Existing files in directory:")
print(existing_files)

# Load data from existing dump files
for i in range(241):  # Assuming 241 time steps
    file_name = f"{file_prefix}{i * timestep_interval}.dump"
    full_path = os.path.join(data_dir, file_name)
    
    if file_name not in existing_files:
        print(f"Skipping missing file: {file_name}")
        continue
    
    print(f"Reading file: {full_path}")
    
    with open(full_path, "r") as file:
        lines = file.readlines()
    
    start_reading = False
    for line in lines:
        if "ITEM: ATOMS" in line:
            start_reading = True
            continue
        
        if start_reading:
            parts = line.split()
            if len(parts) == 6:
                x, y, z, temp = map(float, parts[2:6])
                t = i * timestep_interval * timestep_size  # Compute actual time
                data_points.append([x, y, z, t, temp])

# Convert data to numpy array
if len(data_points) == 0:
    raise ValueError("No data was loaded from dump files. Please check file paths and formats.")
exp_data = np.array(data_points)

# Ensure exp_data is 2D
if exp_data.ndim == 1:
    raise ValueError("Data array is 1D but expected 2D. Check if data is properly loaded.")

Existing files in directory:
{'coord13700000.dump', 'coord112600000.dump', 'coord122100000.dump', 'coord13500000.dump', 'coord121400000.dump', 'coord112900000.dump', 'coord120300000.dump', 'coord18300000.dump', 'coord18900000.dump', 'coord116200000.dump', 'coord123600000.dump', 'coord18000000.dump', 'coord19400000.dump', 'coord118300000.dump', 'coord112800000.dump', 'coord113800000.dump', 'coord112200000.dump', 'coord116900000.dump', 'coord114100000.dump', 'coord110100000.dump', 'coord1400000.dump', 'coord112700000.dump', 'coord115300000.dump', 'coord110300000.dump', 'coord17000000.dump', 'coord117000000.dump', 'coord19500000.dump', 'coord114300000.dump', 'coord17400000.dump', 'coord123100000.dump', 'coord115700000.dump', 'predicted_temperatures.csv', 'coord119900000.dump', 'coord19700000.dump', 'coord11500000.dump', 'coord110200000.dump', 'myfile_helix.in', 'coord111200000.dump', 'coord119700000.dump', 'coord14100000.dump', 'coord113400000.dump', 'coord116600000.dump', 'coord115800000

In [10]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import scipy.optimize
import matplotlib.pyplot as plt
import os

# Directory containing the dump files
data_dir = "/Relax_Al_helix_1_1"
file_prefix = "coord"
num_files = 241  # Number of dump files
timestep_interval = 100000  # Time step between each file
timestep_size = 1.25e-10  # Each step duration in seconds

# Load data from multiple dump files
data_points = []

for i in range(num_files):
    file_name = f"{data_dir}{file_prefix}{i * timestep_interval}.dump"
    if not os.path.exists(file_name):
        continue
    with open(file_name, "r") as file:
        lines = file.readlines()
    
    start_reading = False
    for line in lines:
        if "ITEM: ATOMS" in line:
            start_reading = True
            continue
        
        if start_reading:
            parts = line.split()
            if len(parts) == 6:
                x, y, z, temp = map(float, parts[2:6])
                t = i * timestep_interval * timestep_size  # Compute actual time
                data_points.append([x, y, z, t, temp])

# Convert data to numpy array
if len(data_points) == 0:
    raise ValueError("No data was loaded from dump files. Please check file paths and formats.")
exp_data = np.array(data_points)

# Ensure exp_data is 2D
if exp_data.ndim == 1:
    raise ValueError("Data array is 1D but expected 2D. Check if data is properly loaded.")

# Define the Physics-Informed Machine Learning (PIML) Model
class PINN(keras.Model):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.model = self.build_model(layers)
    
    def build_model(self, layers):
        model = keras.Sequential()
        model.add(keras.layers.InputLayer(input_shape=(4,)))  # (x, y, z, t)
        for width in layers[1:-1]:
            model.add(keras.layers.Dense(width, activation='swish'))
        model.add(keras.layers.Dense(layers[-1], activation=None))  # Output T
        return model
    
    def call(self, x):
        return self.model(x)

# Training data (using extracted experimental data)
x_train, y_train, z_train, t_train, T_real = exp_data[:, 0], exp_data[:, 1], exp_data[:, 2], exp_data[:, 3], exp_data[:, 4]

# Convert to TensorFlow tensors
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
z_train = tf.convert_to_tensor(z_train, dtype=tf.float32)
t_train = tf.convert_to_tensor(t_train, dtype=tf.float32)
T_real = tf.convert_to_tensor(T_real, dtype=tf.float32)

# Instantiate and compile the model
pinn = PINN(layers=[4, 150, 150, 150, 150, 150, 1])  # Increased neurons for better learning
optimizer = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)

# Custom training loop using real data
def train_step():
    with tf.GradientTape() as tape:
        inputs = tf.stack([x_train, y_train, z_train, t_train], axis=1)
        T_pred = pinn(inputs)
        loss = tf.reduce_mean(tf.square(T_pred - T_real))  # Loss using experimental labels
    gradients = tape.gradient(loss, pinn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, pinn.trainable_variables))
    return loss

# Train the model
epochs = 50000  # Increased training epochs
for epoch in range(epochs):
    loss = train_step()
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Save the trained model
pinn.save_weights('piml_heat_slm_weights.h5')

# Load and use the model for prediction
def predict_temperature(x, y, z, t):
    input_tensor = tf.convert_to_tensor([[x, y, z, t]], dtype=tf.float32)
    return float(pinn(input_tensor).numpy()[0])

# Predict temperature and visualize
def predict_and_visualize():
    X, Y, T_values = [], [], []
    for data in exp_data:
        x, y, z, t, T_exp = data
        T_pred = predict_temperature(x, y, z, t)
        X.append(x)
        Y.append(y)
        T_values.append(T_pred)
    
    plt.figure(figsize=(6,5))
    plt.scatter(X, Y, c=T_values, cmap='jet')
    plt.colorbar(label='Predicted Temperature (K)')
    plt.xlabel('X Position (m)')
    plt.ylabel('Y Position (m)')
    plt.title('Predicted Temperature Distribution over Time')
    plt.show()

# Example visualization
predict_and_visualize()

ValueError: No data was loaded from dump files. Please check file paths and formats.